<a href="https://colab.research.google.com/github/AnupJoseph/Knowledge-Graph/blob/master/Graph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
pip install python-igraph

     |████████████████████████████████| 3.2MB 72kB/s 


In [0]:
import igraph

In [0]:
class SearchQueryGraph(object):

  """
    SearchQueryGraph contains methods for implementing the knowledge graph of search terms using the course clicks on a
    search term as historical data required for calculating the various metrics according to the research paper 
    Using Probabilistic Tag Modeling to Improve Recommendations Gokkaya et al August 2017 
    (http://ml4ed.cc/attachments/GokkayaUsing.pdf)
  """
    def __init__(self,data=None):

      self.tag_list=[] # List of Tags required for creating knowledge graph
      self.num_of_clusters = 0 
      
      # Checks for initialising and creating graph and clusters
      if(data == None):
        self.graph = igraph.Graph()
        self._create_clusters()
      else:
        self._create_graph(data)
        self._create_clusters()
      
      
    def _create_graph(self, data):
      """
        The data arguement takes two-tiered list with each inner list being of the form (node1,node2,weight).
        The _create_graph method creates a graph using igraph library and loads it into the object variable self.graph
      """
      self.graph = igraph.Graph.TupleList(data, directed = False, edge_attrs = ['weight'])
        
        
    def _create_clusters(self):
      """
        The _create_clusters method applies the walktrap graph community detection algorithm on the weighted graph 
        of the variable self.graph and creates the clusters obtained from the walktrap algritm on the object attribute 
        self.clusters
      """
      self.tag_list=[]  
      self.graph.layout_fruchterman_reingold()
      self.dendogram = self.graph.community_walktrap(weights=self.graph.es["weight"], steps = 5)
      self.num_of_clusters = self.dendogram.optimal_count
      self.clusters= self.dendogram.as_clustering()

    def _get_subgraph(self,cluster_id):
      """
        Returns subgraphs from the original graph according to the provided cluster_id
      """
      return self.clusters.subgraph(cluster_id)

    def _get_subgraphs(self):
      """
        Yields each subgraph of the graph containing a cluster one-by-one on each of the function call
      """
      for cluster_id in range(self.num_of_clusters):
        yield self.clusters.subgraph(cluster_id)

    def _get_tag_subgraph(self,subgraph,scores=False):
      """
        Returns the tag assigned to the given subgraph according to pagerank algorithm if scores arguement is set to false if set to False.
        
        Returns the tag assisgned to the given subgraph and the pagerank score for each node in the subgraph in the 
        format (node,score) if the scores arguement is set to True
      """
      #Applies pagerank algorithm on the given subgraph.
      pagerank_score = subgraph.pagerank()

      #Finds the tag with the maximum pagerank score in the given subgraph.
      tag_index = pagerank_score.index(max(pagerank_score))
      tag = subgraph.vs[tag_index]["name"]
      if(scores == False):
        return tag

      # Find the pagerank score of each node in the subgraph and assigned to cluster_score
      cluster_score=[]
      for index,node in enumerate(subgraph.vs):
        cluster_score.append([subgraph.vs[index]['name'],pagerank_score[index]])
      return tag, cluster_score

    def get_tags(self,scores=False):
      """
        Yields the tag assigned to each subgraph in the graph if scores is set to False.

        Yields the tag assigned to each subgraph and the pagerank score of each node in the subgraph if scores is set 
        to True
      """
      for subgraph in self._get_subgraphs():
        yield self._get_tag_subgraph(subgraph,scores)

    def get_tag_list(self):

      """
        Returns list of tags assigned to each subgraph in the graph according to the pagerank score of each cluster
      """
      if not self.tag_list:
        for cluster_id in range(self.num_of_clusters):
          tag = self._get_tag_subgraph(self._get_subgraph(cluster_id))
          self.tag_list.append(tag)
      return self.tag_list
    


In [0]:
import pandas as pd

class Search():
    """
      Class search containg the preprocessing and scaffolding functionality required for creating the knowledge graph
      It is a collection of utilities required for creating  
    """
    def __init__(self, filename=None):
        #get list of unique queries
        """
          Reads the filename provided and initialises the dataframe for processing the knowledge graph.  
        """
        self.query_list=[]
        self.total_count=0 #len of query list
        self.filename = filename
        self._term_col = 'search_term'
        self._course_col = 'course_name'
        self._user_col = 'user_ID'
        if(filename == None):
            self.data=pd.DataFrame(column=['user_id','search_term','course_id'])
        else:
            self._read_file()
      
    def _read_file(self):
        """
          Reads the provided csv file into the data attribute.Also creates a list of unique search terms present
          and total count of the search terms
        """
        ##csv self.filename
        self.data =  pd.read_csv(self.filename,na_filter='NaN',na_values='NaN')
        self.query_list = self.data[self._term_col].unique()
        self.total_count = len(self.query_list)
             
    def get_queries(self):
        """
          Returns list of all unique queries in the search space
        """
        return self.query_list

    def get_users(self):
        """
          Returns list of all unique users in the csv file under consideration
        """
        return self.data[self._user_col].unique()
    
    def get_courses_clicked(self):
        """
          Returns a list of click data for all the unique courses in the search space
        """
        return  self.data[self._course_col].unique()
        
    def _get_clicked_courses_by_query(self,query_id):
        """
          Returns the list of unique course clicked on the given search query
        """
        query_data = self.data[self.data[self._term_col]==self.query_list[query_id]]
        return query_data[self._course_col].unique()

    def get_clicked_courses_by_users(self,user_id):
        """
          Returns the list of unique course clicked on the given search query
        """
        user_data = self.data[self.data[self._user_col]==user_id]
        return user_data[self._course_col].unique()
    ''' 
    def _get_clicked_courses_by_query_with_count(self,query_id):
        query_data = self.data[self.data[self._term_col]==self.query_list[query_id]]
        return len(query_data),query_data[self._course_col].value_counts()
    '''
    
    def _get_course_click_count_by_query(self,course_id,query):
        """
          Returns the total number of clicks upon the query and the number of times the specified course was clicked after 
          the given search query
        """
        query_data = self.data[self.data[self._term_col]== query]
        clicks = query_data[query_data[self._course_col]==course_id]
        return len(query_data),len(clicks)

    def _get_course_click_count_by_user(self,course_id,user_id):
        """
          Returns the total number of clicks by the given user and the number of times the specified course was clicked 
          by the given user
        """
        user_data = self.data[self.data[self._user_col]== user_id]
        clicks = user_data[user_data[self._course_col]==course_id]
        return len(user_data),len(clicks)
        
    def get_similarity(self,query1_id,query2_id):
        """
          Returns the Jaccard similarity coefficient between two provided search terms 
        """
        def jaccard_similarity():
            """
              Jaccard similarity in this instance is defined as the size of intersection set of courses clicked after
              the specified search terms divided by the size of union set of courses clicked after the specified search
              term.  
            """
            s1 = set(self._get_clicked_courses_by_query(query1_id))
            s2 = set(self._get_clicked_courses_by_query(query2_id))
            if(len(s1.union(s2))==0):
              return 0
            return float(len(s1.intersection(s2)) / len(s1.union(s2)))
        return jaccard_similarity()
        
    def get_edges(self):
        """
          Yields all the non-zero edges in the graph in the format (start_node,end_node,weight)
        """
        for query1_id in range(0,self.total_count):
            for query2_id in range(0,self.total_count):

                # Find the required search terms for calculating similarity
                vertex1 = self.query_list[query1_id]
                vertex2 = self.query_list[query2_id]

                #Calculate the jaccrard similarity between the two search terms
                edge_weight = self.get_similarity(query1_id,query2_id)
                if(edge_weight != 0):
                    yield vertex1, vertex2, edge_weight
    
    def get_course_query_prob(self,course_id,query):
        """
          Returns the probability of a course been clicked upon the given search term according to historical data
        """
        total_clicks, course_clicks =  self._get_course_click_count_by_query(course_id,query)
        if(total_clicks == 0):
          return 0
        return float(course_clicks / total_clicks)

    def get_user_course_prob(self,user_id,course_id):
        """
          Returns the probability of a course been clicked upon by the given user according to historical data
        """
        total_clicks, course_clicks = self._get_course_click_count_by_user(course_id,user_id)
        if(total_clicks == 0):
          return 0
        return float(course_clicks / total_clicks)
        

In [0]:
s = Search('new_search (1).csv')

In [0]:
list=[]
for edge in s.get_edges():
  list.append(edge)

In [0]:
g = SearchQueryGraph(list)

In [0]:
tags = g.get_tag_list()

In [0]:
courses = s.get_courses_clicked()

In [0]:
users = s.get_users()

In [0]:
t = g.get_tags(scores = True)

In [0]:
g.num_of_clusters

35

In [0]:
def P_Product_Tag(c):
  """
    Returns a lists of the probability of each cluster tag being assigned to the given course
  """
  for tag,query_list in g.get_tags(scores=True):
    #Find the probablity score of a particular cluster tag
    prob = 0
    for query,score in query_list:
        
        prob = prob + (s.get_course_query_prob(c,query)*score)
    if(prob>0):
      
      print("TAG",tag,prob)

In [0]:
course_tag_profile = pd.DataFrame(index = courses,columns = tags)
course_tag_profile

In [0]:
course_tag_profile = pd.DataFrame(index = courses,columns = tags)
for c in courses:
    #Lists of the probability of each cluster tag being assigned to each course
  print("*********************************************************")
  print("COURSE",c)
  for tag,query_list in g.get_tags(scores=True):
    prob = 0
    for query,score in query_list:
        
        prob = prob + (s.get_course_query_prob(c,query)*score)
    course_tag_profile[tag][c] = prob
    if(prob>0):
      print("TAG",tag,prob)

*********************************************************
COURSE Advance java
TAG Advance java 1.0
*********************************************************
COURSE C++
TAG C++ Programming 1.0
*********************************************************
COURSE Personality Development
TAG Personality Development and Communication 1.0
*********************************************************
COURSE Introduction to HTML
TAG Introduction to HTML 0.75
*********************************************************
COURSE MySQL
TAG MySQL 1.0
*********************************************************
COURSE Soft Skills
TAG Soft Skills 1.0
*********************************************************
COURSE Learn Ethical Hacking From Scratch
TAG Learn Ethical Hacking From Scratch 1.0
*********************************************************
COURSE Web designing without code
TAG Web designing without code 1.0
*********************************************************
COURSE Complete Guitar System
TAG Complete 

In [0]:
course_tag_profile.loc[:,:]

In [0]:
import numpy as np
for c1 in courses:
  # Recomendation system for finding out the courses similar to a given course by finding out the inner product of 
  # courses 
  print("****************************************************")
  print("COURSE:c1",c1)
  for c2 in courses:
    if(np.inner(np.array(course_tag_profile.loc[c1,:]),np.array(course_tag_profile.loc[c2,:]))>0):
      print("SIMILAR COURSES", c2)

****************************************************
COURSE:c1 Advance java
SIMILAR COURSES Advance java
****************************************************
COURSE:c1 C++
SIMILAR COURSES C++
****************************************************
COURSE:c1 Personality Development
SIMILAR COURSES Personality Development
****************************************************
COURSE:c1 Introduction to HTML
SIMILAR COURSES Introduction to HTML
SIMILAR COURSES html
****************************************************
COURSE:c1 MySQL
SIMILAR COURSES MySQL
****************************************************
COURSE:c1 Soft Skills
SIMILAR COURSES Soft Skills
****************************************************
COURSE:c1 Learn Ethical Hacking From Scratch
SIMILAR COURSES Learn Ethical Hacking From Scratch
****************************************************
COURSE:c1 Web designing without code
SIMILAR COURSES Web designing without code
****************************************************
COURSE:c1 

In [0]:
for u in users:
  print("*************************************************************")
  print("User",u)
  for tag,query_list in g.get_tags(scores=True):
    #print("Tag",tag)
    prob_UT = []

    # Probablity of course being assigned a particular tag
    for c in s.get_clicked_courses_by_users(u):

      #Calculate the probabilty of a user clicking the course
      prob_UC = s.get_user_course_prob(u,c)
      #print("P(U|C)",prob_UC)

      # Calculate the probabilty of the course being clicked after a search term
      prob = 0
      for query,score in query_list:
          prob = prob + (s.get_course_query_prob(c,query)*score)

      # Computing the probabilty of user being assigned the tag as sum of the above calculated values 
      prob_UTi = prob_UC * prob
      #print("P(C|T)",prob)
      if(prob_UTi>0):
          #print("P(U|T)for i course",prob_UTi,c)
          prob_UT.append(prob_UTi)
    if(sum(prob_UT)>0):
      print("Tag",tag)
      print("P(U|T)=",sum(prob_UT))



 
      

*************************************************************
User 10000282
Tag Advance java
P(U|T)= 0.05555555555555555
Tag C++ Programming
P(U|T)= 0.05555555555555555
Tag Personality Development and Communication
P(U|T)= 0.05555555555555555
Tag Introduction to HTML
P(U|T)= 0.041666666666666664
Tag MySQL
P(U|T)= 0.05555555555555555
Tag Soft Skills
P(U|T)= 0.05555555555555555
Tag Learn Ethical Hacking From Scratch
P(U|T)= 0.05555555555555555
Tag Web designing without code
P(U|T)= 0.05555555555555555
Tag Complete Guitar System
P(U|T)= 0.05555555555555555
Tag Hadoop
P(U|T)= 0.05555555555555555
Tag Teacher Traning
P(U|T)= 0.05555555555555555
Tag Fashion Designing
P(U|T)= 0.05555555555555555
Tag Blockchain
P(U|T)= 0.05555555555555555
Tag Robotics
P(U|T)= 0.05555555555555555
Tag Master Web Design in Photoshop
P(U|T)= 0.05555555555555555
Tag Astrobiology
P(U|T)= 0.05555555555555555
Tag Data visualization
P(U|T)= 0.032407407407407406
Tag IOT (Level 2   certification course
P(U|T)= 0.055555555

In [0]:
s.get_clicked_courses_by_users(501429)

array([], dtype=object)

In [0]:
from igraph import *
g = Graph()
g.add_vertices(15)
g.add_edges([(1, 0),(3, 4),(10, 3),(5, 6),(9, 4)])
score = g.pagerank()
score.index(max(score))


3

In [0]:
list

In [0]:
q_graph = SearchQueryGraph(list)
q_graph.create_clusters()

In [0]:
tag = q_graph.get_tags()

In [0]:
taglist =[]
for tag in q_graph.get_tags():
  print("Tag :",tag)
  taglist.append(tag)
pagerank_score=[]

for tag,score in q_graph.get_tags(scores=True):
  print("Tag :",tag)
  print("Cluster:",score)
  